# Mask RCNN and U-Net Model
In the following notebook, we create a model based on mask rcnn (https://github.com/akTwelve/Mask_RCNN/tree/d9f01dcd83b1e90e5ea0b4b6d5ba753770e114ac).
However, it will be combined with U-Net. In this way, we hope to combine the strenghts of both model and improve the instance segmentation of C Elegans.
To do so, we will use the Encoder of U-Net as the Backbone of Mask RCNN instead of ResNet. This part is responsible for extraxting features out of images.


In [2]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

Matplotlib is building the font cache; this may take a moment.


In [4]:
import torch
import torch.nn as nn
import segmentation_models as sm

class UNetEncoder(tf.keras.Model):
    def __init__(self, backbone_name='resnet34', encoder_weights='imagenet'):
        super(UNetEncoder, self).__init__()
        self.unet = sm.Unet(backbone_name=backbone_name, encoder_weights=encoder_weights)
    
    def call(self, inputs):
        # Durchlaufe das Modell und extrahiere die Encoder-Feature-Maps
        encoder_outputs = self.unet.encoder(inputs)
        return encoder_outputs

AttributeError: module 'keras.utils' has no attribute 'generic_utils'

In [3]:
# root directory of the project
ROOT_DIR = '/content/gdrive/MyDrive/'

import mrcnn.utils
import mrcnn.visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import mrcnn.CElegansDataset

%matplotlib inline

COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mrcnn/mask_rcnn_coco.h5")
from mrcnn.cElegans import CElegansConfig
config = CElegansConfig()
CELEGANS_DIR = os.path.join(ROOT_DIR, "dataset")

# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.

    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

from mrcnn.cElegans import CElegansDataset

# Load Training dataset
dataset = CElegansDataset()
dataset.load_c_elegans(CELEGANS_DIR, "train")
dataset.prepare()

# Load validation dataset
datatset_val = CElegansDataset()
datatset_val.load_c_elegans(CELEGANS_DIR, "val")
datatset_val.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))
print("Images: {}\nClasses: {}".format(len(datatset_val.image_ids), datatset_val.class_names))

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mrcnn/mask_rcnn_coco.h5")

# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset, datatset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=1,
            layers='heads')

# Fine tune all layers
# Passing layers="all" trains all layers. You can also
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset, datatset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2,
            layers="all")

ModuleNotFoundError: No module named 'scipy'